<a href="https://colab.research.google.com/github/jinchen1036/Product-Price-Prediction/blob/main/experiment/AttributeEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
# read sample data
sample_df = pd.read_csv("https://raw.githubusercontent.com/jinchen1036/Product-Price-Prediction/main/data/sample_data.csv")
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   train_id           1000 non-null   int64  
 1   name               1000 non-null   object 
 2   item_condition_id  1000 non-null   int64  
 3   category_name      996 non-null    object 
 4   brand_name         561 non-null    object 
 5   price              1000 non-null   float64
 6   shipping           1000 non-null   int64  
 7   item_description   1000 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 62.6+ KB


In [4]:
# drop row with missing value  as it not need in this task
sample_df.dropna(inplace=True)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558 entries, 6 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   train_id           558 non-null    int64  
 1   name               558 non-null    object 
 2   item_condition_id  558 non-null    int64  
 3   category_name      558 non-null    object 
 4   brand_name         558 non-null    object 
 5   price              558 non-null    float64
 6   shipping           558 non-null    int64  
 7   item_description   558 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 39.2+ KB


## Number of Categories for `brand_name` and `category_name`

In [10]:
print("Number of categories for category_name attribute is ",len(sample_df.category_name.unique()))

Number of categories for category_name attribute is  183


In [13]:
for category in sample_df.category_name.unique()[:5]:
  print(category)

Women/Other/Other
Women/Tops & Blouses/Blouse
Women/Shoes/Sandals
Electronics/Cell Phones & Accessories/Cell Phones & Smartphones
Women/Tops & Blouses/Tunic


In [16]:
print("Number of categories for brand_name attribute is ",len(sample_df.brand_name.unique()))

Number of categories for brand_name attribute is  233


In [17]:
for brand_name in sample_df.brand_name.unique()[:5]:
  print(brand_name)

Victoria's Secret
Mossimo
Express
HTC
Disney Collection


## One Hot Encoding